In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import glob
train_list = glob.glob("/kaggle/input/sanskrit-wikipedia-articles/train/train/*.txt")
valid_list = glob.glob("/kaggle/input/sanskrit-wikipedia-articles/valid/valid/*.txt")

In [3]:
len(train_list)

18812

In [4]:

#readig and appending all small files to single Train and Valid files
with open("/kaggle/working/full.txt", "wb") as outfile:
    for f in train_list:
        with open(f, "rb") as infile:
            outfile.write(infile.read())
            outfile.write(b"\n\n")

# Remove the line with the problematic file "sa.txt"
#with open("/content/sa.txt", "rb") as infile:
#        outfile.write(infile.read())

with open("/kaggle/working/full_val.txt", "wb") as outfile:
    for f in valid_list:
        with open(f, "rb") as infile:
            outfile.write(infile.read())
            outfile.write(b"\n\n")


In [5]:
import re

with open("/kaggle/working/full_val.txt", "r", encoding="utf-8") as infile:
    text = infile.read()



In [6]:
len(text)

24593450

In [7]:
# Remove all numbers from the text
text = re.sub(r'\d+', '', text)

In [8]:
len(text)

20935352

In [9]:
text[10000]

'ळ'

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 15000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)



In [11]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [12]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [14]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [15]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [16]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [17]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

11.175736 M parameters


In [18]:

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 6.4455, val loss 6.4393
step 500: train loss 1.9274, val loss 1.9760
step 1000: train loss 1.5392, val loss 1.5751
step 1500: train loss 1.3725, val loss 1.4189
step 2000: train loss 1.2826, val loss 1.3207
step 2500: train loss 1.2053, val loss 1.2469
step 3000: train loss 1.1412, val loss 1.1960
step 3500: train loss 1.1118, val loss 1.1668
step 4000: train loss 1.1025, val loss 1.1387
step 4500: train loss 1.0649, val loss 1.1140
step 5000: train loss 1.0437, val loss 1.0993
step 5500: train loss 1.0281, val loss 1.0781
step 6000: train loss 1.0276, val loss 1.0757
step 6500: train loss 1.0073, val loss 1.0602
step 7000: train loss 0.9919, val loss 1.0371
step 7500: train loss 0.9778, val loss 1.0327
step 8000: train loss 0.9716, val loss 1.0145
step 8500: train loss 0.9647, val loss 1.0177
step 9000: train loss 0.9484, val loss 0.9953
step 9500: train loss 0.9358, val loss 0.9955
step 10000: train loss 0.9402, val loss 0.9894
step 10500: train loss 0.9381, val lo

In [19]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=5000)[0].tolist()))
open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

	मांचे चेत् “ व स्वर्गे पर्याप्तम् ।येषां प्रमो भवः प्रयोजनम् । प्रथमप्रपाते, प्रतिभां वर्णयत् तदपि त्वं सर्वात्मकनामिना भवेत् । मांचे दृढमनामके अनुद्दितेऽपि संस्कृते निष्क्षरार्थं सार्थसंख्यकार्ये जनानपि । क्वापि गुरोः मां, प्रतिभा च दृश्यते । अनेन रोगाः इतेषां मण्डनमेव शुष्को भवेत् चेत्, महतिभेदेन मम स्वरोऽपि इत्यर्थः ।
अशोषणत्रयं त्वमपि पश्यति भारद्वये । उपर्युक्तताम्, हरिहरोन्भयोः इति काश्मीरः ‘रोगमाय न्यूनं हरति ।।' इदमुपरि व्यज्ञानस्य महाकवीनां सर्वाक्षीकरणम् उपयुज्य अधीनं प्रोक्ततां न भरतं न्यूनत्वेन अस्ति। रतिः इस्लामिय वृतबुद्धिः, अदिक्लुषा च । प्रसादेभ्यच्च वृतबह्वीं सद्यः नाभरणयुक्तः । अस्य तात्विकंतु पञ्च मयूगः सन्दर्भं विनैश्चत्वारो अस्त्यसः।।
न दिदीम्बवर्य अधीत्येष्वात्यथा।।
विशिष्टपदस्य संयोगेन च उपदेष्टतृण परमातृवृत्ति:।।।
)	तमुवाह - विश्वम् इत्युक्त्वापरममूलगच्छ्नतु न विशिष्टपूजासु तदन्यस्य यदृच्छता भूमेः पञ्च मुक्ता इत्ययं भविष्यति। यथा सहञ्चितः छ – न भूतपदार्थस्य स्याद् भूतमासीत् । तस्य विषये विशिष्टासु सभाजस्य..... तः  पं.   पिनाङ्के नागनर् परित्यज्येत् ।
नियमत्रयमि

10001

In [20]:
torch.save(model, 'gpt_language_model.pt')

In [21]:
torch.save(model.state_dict(), 'gpt_language_model_state_dict.pt')